In [3]:
pip install nltk

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached regex-2024.11.6-cp313-cp313-win_amd64.whl.metadata (41 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached regex-2024.11.6-cp313-cp313-win_amd64.whl (273 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
#Import libraries
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
import pickle

In [5]:
#Download NLTK data files
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\William\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\William\AppData\Roaming\nltk_data...


True

In [19]:
#Import datasets
twitter = pd.read_csv('raw data\\twitter_training.csv')
emotion_text = pd.read_csv('raw data\\tweet_emotions.csv')
youtube_comments = pd.read_csv('raw data\\YoutubeCommentsDataSet.csv')

In [20]:
#Clean data (duplicates, missing values, uniform text, etc.)
def clean_text(text):
    if isinstance(text, str):
        # Convert to lowercase
        text = text.lower()
        # Remove URLs
        text = re.sub(r'http\S+|www\S+|https\S+', '', text)
        # Remove user mentions and hashtags
        text = re.sub(r'@\w+|#\w+', '', text)
        # Remove punctuation and special characters
        text = re.sub(r'[^\w\s]', '', text)
        # Remove numbers
        text = re.sub(r'\d+', '', text)
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text).strip()
        
        # Remove stopwords and lemmatize
        stop_words = set(stopwords.words('english'))
        lemmatizer = WordNetLemmatizer()
        words = [lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words]
        
        return ' '.join(words)
    return ''

In [22]:
#Map emotions to uniform data
emotion_to_sentiment = {
    'enthusiasm': 'positive',
    'surprise': 'positive',
    'love': 'positive',
    'fun': 'positive',
    'happiness': 'positive',
    'neutral': 'neutral',
    'relief': 'positive',
    'anger': 'negative',
    'boredom': 'negative',
    'hate': 'negative',
    'worry': 'negative',
    'sadness': 'negative',
    'empty': 'negative'
}

# Apply mapping to emotion dataset
emotion_text['sentiment'] = emotion_text['sentiment'].map(emotion_to_sentiment)

In [18]:
twitter.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74681 entries, 0 to 74680
Data columns (total 4 columns):
 #   Column                                                 Non-Null Count  Dtype 
---  ------                                                 --------------  ----- 
 0   2401                                                   74681 non-null  int64 
 1   Borderlands                                            74681 non-null  object
 2   Positive                                               74681 non-null  object
 3   im getting on borderlands and i will murder you all ,  73995 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB


In [16]:
#Preprocess data
# Twitter dataset
twitter['cleaned_text'] = twitter['Tweet content'].apply(clean_text)
twitter_subset = twitter[['cleaned_text', 'sentiment']]

# YouTube dataset
youtube_comments['cleaned_text'] = youtube_comments['Comment'].apply(clean_text)
youtube_subset = youtube_comments[['cleaned_text', 'Sentiment']]
youtube_subset = youtube_subset.rename(columns={'Sentiment': 'sentiment'})

# Emotion dataset
emotion_text['cleaned_text'] = emotion_text['content'].apply(clean_text)
emotion_subset = emotion_text[['cleaned_text', 'sentiment']]

KeyError: 'Tweet content'

In [ ]:
#Concat data into one dataset
combined_data = pd.concat([twitter_subset, youtube_subset, emotion_subset], ignore_index=True)
combined_data = combined_data.dropna()

In [ ]:
#Encode data
label_encoder = LabelEncoder()
combined_data['sentiment_encoded'] = label_encoder.fit_transform(combined_data['sentiment'])


In [ ]:
# Print the mapping for reference
sentiment_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Sentiment mapping:", sentiment_mapping)

In [ ]:
#Save data
combined_data.to_csv('combined_sentiment_data.csv', index=False)

In [ ]:
#Save the label encoder for future use
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)